In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<Axes: xlabel='count', ylabel='domain'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

<Axes: xlabel='count', ylabel='posted_by'>

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

/tmp/ipykernel_673784/2178239382.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby(["domain", "posted_by"])


domain         \
                                            count unique   
domain               posted_by                             
americanthinker.com  MtnClimber               104      1   
                     Red Badger                 1      1   
                     SeekAndFind               38      1   
                     george76                   1      1   
babylonbee.com       E. Pluribus Unum           3      1   
                     Morgana                    1      1   
breitbart.com        Red Badger                 8      1   
                     george76                   4      1   
                     E. Pluribus Unum           2      1   
                     Morgana                    2      1   
                     ChicagoConservative27    517      1   
cbsnews.com          Red Badger                 4      1   
                     george76                   1      1   
                     Morgana                    1      1   
                     ChicagoConservative27      4      1   
                     DallasBiff                 3      1   
                     nickcarraway              12      1   
dailymail.co.uk      MtnClimber                 1      1   
                     Red Badger                 3      1   
                     george76                   4      1   
                     Morgana                   60      1   
                     ChicagoConservative27      3      1   
                     nickcarraway               1      1   
                     Libloather                22      1   
foxnews.com          Red Badger                 1      1   
                     SeekAndFind                1      1   
                     george76                   3      1   
                     Morgana                    3      1   
                     ChicagoConservative27      3      1   
                     DallasBiff                 4      1   
                     nickcarraway               3      1   
                     Libloather                 6      1   
frontpagemag.com     MtnClimber                 4      1   
                     george76                   3      1   
                     E. Pluribus Unum           3      1   
justthenews.com      Red Badger                 3      1   
                     george76                   2      1   
                     E. Pluribus Unum           4      1   
msn.com              SeekAndFind                9      1   
                     E. Pluribus Unum           1      1   
                     ChicagoConservative27      2      1   
                     DallasBiff                13      1   
                     nickcarraway              11      1   
                     Libloather                 1      1   
nbcnews.com          Red Badger                 5      1   
                     SeekAndFind                1      1   
                     DallasBiff                 6      1   
                     nickcarraway               1      1   
newsweek.com         MtnClimber                 1      1   
                     Red Badger                 3      1   
                     george76                   1      1   
                     DallasBiff                 8      1   
                     nickcarraway               4      1   
                     Libloather                 1      1   
nypost.com           MtnClimber                 1      1   
                     Red Badger                 3      1   
                     SeekAndFind                3      1   
                     george76                  13      1   
                     ChicagoConservative27    124      1   
                     DallasBiff                 7      1   
                     nickcarraway              58      1   
                     Libloather                86      1   
nytimes.com          E. Pluribus Unum         121      1   
                     DallasBiff                 2      1   
pjmedia.com          Re

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

/tmp/ipykernel_673784/1921096508.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby('title').


timestamp  \
title                                                                    
Sounds like Kamala is loaded for “Baier” tonigh... 2024-10-17 00:51:23   
Donald Trump groped me in what felt like a ‘twi... 2024-10-24 00:52:35   

                                                    replies  
title                                                        
Sounds like Kamala is loaded for “Baier” tonigh...    182.0  
Donald Trump groped me in what felt like a ‘twi...    157.0